In [ ]:
import sys
sys.path.append('..')

from src.data_loader import download_stock_data, get_data_info
from src.strategy import calculate_sma, generate_signals

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('seaborn_v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14,7)


In [ ]:
ticker = "RELIANCE.NS"
df = download_stock_data(ticker, period ="5y", interval ="1d")

if df is not None:
    print(f" Downloading {ticker} stock data...")
    print(f"period: {df.index[0].date()} to {df.index[-1].date()}")
else:
    print("Data download failed")
    